In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import covid_19_shared


pop = covid_19_shared.USA_population()
# print("%-30s\t%-10s\t%-17s\t%-10s\t%-17s" 
print("%-30s%-10s%-20s%-10s%-16s" 
      % ("State", "New Conf.", "Conf./Popul.(%)"
         , "New Death", "Death/Popul.(%)")
     )

states = covid_19_shared.list_of_JHUD_US_states()
# df = pd.DataFrame({"State" : states, "Statistics" : [None] * len(states)})
# df.set_index("State")

df = pd.DataFrame()

for state in states:
    if state in pop:
        
        stats = covid_19_shared.USA_State_COVID19_Stats(state)
        state_pop = int(pop[state])
        stats.pops_ratio_conf_ = stats.cv_data_.Confirmed * 100 / state_pop
        stats.pops_ratio_deaths_ = stats.cv_data_.Dead * 100 / state_pop
        print("%-30s%-10d%-20f%-10d%-16f" 
              % (state
                    , int(stats.confirmed_new_[-1])
                    , stats.pops_ratio_conf_[-1]
                    , int(stats.deaths_new_[-1])
                    , stats.pops_ratio_deaths_[-1]
                   )
             )
        df = df.append(pd.DataFrame({"State" : [state], "Statistics": [stats], "Population" : [state_pop]}))
#         stats.dynamics()

df = df.set_index("State")
# print(df)

State                         New Conf. Conf./Popul.(%)     New Death Death/Popul.(%) 
Alabama                       1205      4.261470            2         0.064522        
Alaska                        520       2.791486            0         0.011827        
Arizona                       1880      4.056059            17        0.096433        
Arkansas                      1038      4.179336            17        0.071504        
California                    3351      2.611033            12        0.048245        
Colorado                      3017      2.604472            5         0.047602        
Connecticut                   0         2.185867            0         0.130690        
Delaware                      345       2.962690            2         0.079961        
District of Columbia          110       2.991576            0         0.108688        
Florida                       6820      4.488501            21        0.091063        
Georgia                       1253      3.8

In [9]:
df["NewConf"] = pd.Series(index=df.index, data=[int(v.confirmed_new_[-1]) for v in df["Statistics"]])
total = df.NewConf.sum()
totalPop = df.Population.sum()
df["NewConfShare"] = df["NewConf"] * 100 / total
df["PopulShare"] =df["Population"] * 100 / totalPop

# Indicator of cases intensity
df["Intencity"] = df.NewConfShare / df.PopulShare
df2 = df.sort_values('Intencity', ascending=False)
df2


,Statistics,Population,NewConf,NewConfShare,PopulShare,Intencity
State,,,,,,
South Dakota,<covid_19_shared.USA_State_COVID19_Stats objec...,814180,1428,1.348315,0.260562,5.174650
North Dakota,<covid_19_shared.USA_State_COVID19_Stats objec...,672591,1101,1.039562,0.215249,4.829581
Iowa,<covid_19_shared.USA_State_COVID19_Stats objec...,3046355,4524,4.271551,0.974923,4.381424
Wyoming,<covid_19_shared.USA_State_COVID19_Stats objec...,563626,713,0.673213,0.180377,3.732258
Oklahoma,<covid_19_shared.USA_State_COVID19_Stats objec...,3751351,4507,4.255500,1.200542,3.544648
Minnesota,<covid_19_shared.USA_State_COVID19_Stats objec...,5303925,5908,5.578321,1.697412,3.286369
Utah,<covid_19_shared.USA_State_COVID19_Stats objec...,2763885,2386,2.252856,0.884524,2.546969
Nebraska,<covid_19_shared.USA_State_COVID19_Stats objec...,1826341,1574,1.486168,0.584483,2.542706
Illinois,<covid_19_shared.USA_State_COVID19_Stats objec...,12830632,10009,9.450477,4.106179,2.301526
